In [ ]:
import itertools
import json
import pickle
import re
import shutil
import os
import subprocess
import sys
from pathlib import Path
from typing import Any, Dict
from functools import partial
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scri
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from spherical_functions import LM_index as lm


# Functions

In [ ]:
def add_uniform_noise(
    input_file, output_file, noise_amp=1e-16, add_relative_noise=False
):
    shutil.copy(input_file, output_file)
    with h5py.File(output_file, "r+") as outfile:
        for key, data in outfile.items():
            if "Version" in key:
                continue
            if isinstance(data, h5py.Dataset):
                print(f"----Modifying {key}")
                if add_relative_noise:
                    data[:, 1:] = data[:, 1:] * (
                        1
                        + noise_amp
                        * np.random.uniform(low=-1, high=1, size=data[:, 1:].shape)
                    )
                else:
                    data[:, 1:] = data[:, 1:] + noise_amp * (
                        np.random.uniform(low=-1, high=1, size=data[:, 1:].shape)
                    )


def remove_values_below_tolerance(
    input_file, output_file, tolerance=1e-16, use_relative_tolerance=False
):
    shutil.copy(input_file, output_file)
    with h5py.File(output_file, "r+") as outfile:
        for key, data in outfile.items():
            if "Version" in key:
                continue
            if isinstance(data, h5py.Dataset):
                print(f"----Modifying {key}")
                data_no_t = data[:, 1:]

                if use_relative_tolerance:
                    max_per_row = np.max(np.abs(data_no_t), axis=1)
                    thresh = (tolerance * max_per_row)[:, np.newaxis]
                else:
                    # absolute threshold
                    thresh = tolerance
                cleaned = np.where(np.abs(data_no_t) < thresh, 0, data_no_t)
                data[:, 1:] = cleaned

def just_link(input_file, output_file):
    os.symlink(input_file, output_file)

def get_diff_WT_data(file1,file2):
    with h5py.File(file1, "r") as f1, h5py.File(file2, "r") as f2:
        vars = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
        diff_data = {}
        for key in vars:
            print(f"----Taking diff: {key}")
            diff_data[key] = np.array(f2[key])
            diff_data[key] = diff_data[key][:,1:] - f1[key][:,1:]
        return diff_data

In [ ]:

def make_config_file(
    BoundaryDataPath: Path,
    InputSavePath: Path = None,
    which_ID: str = "ConformalFactor",
    options_dict_user={},
    observer_vol_data = False,
) -> Path:
    options_dict = {
        "Cce.Evolution.TimeStepper.AdamsBashforth.Order": 3,
        "Cce.Evolution.StepChoosers.Constant": 0.1,
        "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).AbsoluteTolerance": 1e-9,
        "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).RelativeTolerance": 1e-7,
        "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).AbsoluteTolerance": 1e-9,
        "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).RelativeTolerance": 1e-8,
        "Cce.LMax": 20,
        "Cce.NumberOfRadialPoints": 15,
        "Cce.ObservationLMax": 8,
        "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MinOrder": 10,
        "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MaxOrder": 10,
        "Cce.H5LookaheadTimes": 10000,
        "Cce.Filtering.RadialFilterHalfPower": 64,
        "Cce.Filtering.RadialFilterAlpha": 35.0,
        "Cce.Filtering.FilterLMax": 18,
        "Cce.ScriInterpOrder": 5,
        "Cce.ScriOutputDensity": 1,
    }

    for key in options_dict_user.keys():
        if key not in options_dict:
            raise ValueError(f"Key {key} is not a valid option.")
        else:
            options_dict[key] = options_dict_user[key]

    CCE_ID_data = ""
    match which_ID:
        case "ConformalFactor":
            CCE_ID_data = """
    ConformalFactor:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000 # Do extra iterations in case we improve.
      RequireConvergence: False # Often don't converge to 1e-13, but that's fine
      OptimizeL0Mode: True
      UseBetaIntegralEstimate: False
      ConformalFactorIterationHeuristic: SpinWeight1CoordPerturbation
      UseInputModes: False
      InputModes: []
"""
        case "InverseCubic":
            CCE_ID_data = """
    InverseCubic:
"""
        case "ZeroNonSmooth":
            CCE_ID_data = """
    ZeroNonSmooth:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000
      RequireConvergence: False
"""
        case "NoIncomingRadiation":
            CCE_ID_data = """
    NoIncomingRadiation:
      AngularCoordTolerance: 1e-13
      MaxIterations: 1000
      RequireConvergence: False
"""
    if InputSavePath is None:
        InputSavePath = BoundaryDataPath.parent / "cce.yaml"
    assert InputSavePath.parent.exists()

    config_file = f"""
# Distributed under the MIT License.
# See LICENSE.txt for details.

# This block is used by testing and the SpECTRE command line interface.
Executable: CharacteristicExtract
Testing:
  Check: parse
  Priority: High

---
Evolution:
  InitialTimeStep: 0.25
  MinimumTimeStep: 1e-7
  InitialSlabSize: 10.0

ResourceInfo:
  AvoidGlobalProc0: false
  Singletons: Auto

Observers:
  VolumeFileName: "vol_{str(InputSavePath.stem)}"
  ReductionFileName: "red_{str(InputSavePath.stem)}"

EventsAndTriggersAtSlabs:
  # Write the CCE time step every Slab. A Slab is a fixed length of simulation
  # time and is not influenced by the dynamically adjusted step size.
  - Trigger:
      Slabs:
        EvenlySpaced:
          Offset: 0
          Interval: 1
    Events:
      - ObserveTimeStep:
          # The output is written into the "ReductionFileName" HDF5 file under
          # "/SubfileName.dat"
          SubfileName: CceTimeStep
          PrintTimeToTerminal: true
    #   - ObserveFields:
    #       VariablesToObserve:
    #         - BondiBeta
    #         - Du(J)
    #         - DuRDividedByR
    #         - Dy(BondiBeta)
    #         - Dy(Du(J))
    #         - Dy(Dy(BondiBeta))
    #         - Dy(Dy(Du(J)))
    #         - Dy(Dy(J))
    #         - Dy(Dy(Q))
    #         - Dy(Dy(U))
    #         - Dy(Dy(W))
    #         - Dy(H)
    #         - Dy(J)
    #         - Dy(Q)
    #         - Dy(U)
    #         - Dy(W)
    #         - EthRDividedByR
    #         - H
    #         - InertialRetardedTime
    #         - J
    #         - OneMinusY
    #         - Psi0
    #         - Psi1
    #         - Q
    #         - R
    #         - U
    #         - W

EventsAndTriggersAtSteps:

Cce:
  Evolution:
    TimeStepper:
      AdamsBashforth:
        Order: {options_dict['Cce.Evolution.TimeStepper.AdamsBashforth.Order']} # Going to higher order doesn't seem necessary for CCE
    StepChoosers:
      - Constant: {options_dict['Cce.Evolution.StepChoosers.Constant']} # Don't take steps bigger than 0.1M
      - LimitIncrease:
          Factor: 2
      - ErrorControl(SwshVars):
          AbsoluteTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(SwshVars).AbsoluteTolerance']}
          RelativeTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(SwshVars).RelativeTolerance']}
          # These factors control how much the time step is changed at once.
          MaxFactor: 2
          MinFactor: 0.25
          # How close to the "perfect" time step we take. Since the "perfect"
          # value assumes a linear system, we need some safety factor since our
          # system is nonlinear, and also so that we reduce how often we retake
          # time steps.
          SafetyFactor: 0.9
      - ErrorControl(CoordVars):
          AbsoluteTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(CoordVars).AbsoluteTolerance']}
          RelativeTolerance: {options_dict['Cce.Evolution.StepChoosers.ErrorControl(CoordVars).RelativeTolerance']}
          # These factors control how much the time step is changed at once.
          MaxFactor: 2
          MinFactor: 0.25
          # How close to the "perfect" time step we take. Since the "perfect"
          # value assumes a linear system, we need some safety factor since our
          # system is nonlinear, and also so that we reduce how often we retake
          # time steps.
          SafetyFactor: 0.9

  # The number of angular modes used by the CCE evolution. This must be larger
  # than ObservationLMax. We always use all of the m modes for the LMax since
  # using fewer m modes causes aliasing-driven instabilities.
  LMax: {options_dict['Cce.LMax']}
  # Probably don't need more than 15 radial grid points, but could increase
  # up to ~20
  NumberOfRadialPoints: {options_dict['Cce.NumberOfRadialPoints']}
  # The maximum ell we use for writing waveform output. While CCE can dump
  # more, you should be cautious with higher modes since mode mixing, truncation
  # error, and systematic numerical effects can have significant contamination
  # in these modes.
  ObservationLMax: {options_dict['Cce.ObservationLMax']}

  InitializeJ:
    # To see what other J-initialization procedures are available, comment
    # out this group of options and do, e.g. "Blah:" The code will print
    # an error message with the available options and a help string.
    # More details can be found at spectre-code.org.
{CCE_ID_data}

  StartTime: Auto
  EndTime: Auto
  ExtractionRadius: Auto

  BoundaryDataFilename: {BoundaryDataPath.name}
  H5Interpolator:
    BarycentricRationalSpanInterpolator:
      MinOrder: {options_dict['Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MinOrder']}
      MaxOrder: {options_dict['Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MaxOrder']}

  H5LookaheadTimes: {options_dict['Cce.H5LookaheadTimes']}

  Filtering:
    RadialFilterHalfPower: {options_dict['Cce.Filtering.RadialFilterHalfPower']}
    RadialFilterAlpha: {options_dict['Cce.Filtering.RadialFilterAlpha']}
    FilterLMax: {options_dict['Cce.Filtering.FilterLMax']}

  ScriInterpOrder: {options_dict['Cce.ScriInterpOrder']}
  ScriOutputDensity: {options_dict['Cce.ScriOutputDensity']}

"""

    with InputSavePath.open("w") as f:
        f.writelines(config_file)

    return InputSavePath


def make_submit_file(
    save_folder_path: Path,
    cce_input_file_path: Path,
    CCE_Executable_path: Path,
    write_scripts_only=False,
):
    submit_script = f"""#!/bin/bash -
#SBATCH -J CCE_{save_folder_path.stem}             # Job Name
#SBATCH -o CCE.stdout                 # Output file name
#SBATCH -e CCE.stderr                 # Error file name
#SBATCH -n 2                          # Number of cores
#SBATCH -p expansion                  # Queue name
#SBATCH --ntasks-per-node 2           # number of MPI ranks per node
#SBATCH -t 24:0:00   # Run time
#SBATCH -A sxs                # Account name
#SBATCH --no-requeue
#SBATCH --reservation=sxs_standing

# Spectre related stuff to load JeMalloc
export SPECTRE_HOME=/central/groups/sxs/hchaudha/spectre
. $SPECTRE_HOME/support/Environments/caltech_hpc_gcc.sh
spectre_load_modules

# Go to the correct folder with the boundary data
cd {save_folder_path}

# run CCE
{CCE_Executable_path} --input-file ./{cce_input_file_path.name}
"""
    submit_script_path = save_folder_path / "submit.sh"
    submit_script_path.write_text(submit_script)

    if not write_scripts_only:
        command = f"cd {save_folder_path} && qsub {submit_script_path}"
        status = subprocess.run(command, capture_output=True, shell=True, text=True)
        if status.returncode == 0:
            print(f"Succesfully submitted {submit_script_path}\n{status.stdout}")
        else:
            sys.exit(
                f"Job submission failed for {submit_script_path} with error: \n{status.stdout} \n{status.stderr}"
            )


def create_CCE_folder(
    NewCCEPath: Path,
    BondiBaseH5File: Path,
    CCE_Executable_path: Path,
    WT_data_modification_function,
    CCE_ID: str = "ConformalFactor",
    options_dict_user={},
    write_scripts_only=False,

):
    NewCCEPath = NewCCEPath.resolve()
    BondiBaseH5File = BondiBaseH5File.resolve()
    CCE_Executable_path = CCE_Executable_path.resolve()

    if not BondiBaseH5File.exists():
        raise Exception(f"{BondiBaseH5File} does not exist!")
    if not CCE_Executable_path.exists():
        raise Exception(f"{CCE_Executable_path} does not exist!")

    NewCCEPath.mkdir(parents=True, exist_ok=False)

    WT_data_modification_function(BondiBaseH5File, NewCCEPath/BondiBaseH5File.name)

    make_config_file(
        BoundaryDataPath=NewCCEPath / BondiBaseH5File.name,
        InputSavePath=NewCCEPath / "cce.yaml",
        which_ID=CCE_ID,
        options_dict_user=options_dict_user,
    )

    make_submit_file(
        save_folder_path=NewCCEPath,
        cce_input_file_path=NewCCEPath / "cce.yaml",
        CCE_Executable_path=CCE_Executable_path,
        write_scripts_only=write_scripts_only,
    )

    print("DONE!\n\n\n\n\n")



In [ ]:
# create_CCE_folder(
#     NewCCEPath=Path(
#         "/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/runs/original_tol_10"
#     ),
#     BondiBaseH5File=Path(
#         "/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/data/BondiCceR0250.h5"
#     ),
#     CCE_Executable_path=Path(
#         "/resnick/groups/sxs/hchaudha/spec_runs/CCE_stuff/noise_in_WT_data/data/CharacteristicExtract"
#     ),
#     WT_data_modification_function=partial(
#         add_uniform_noise, noise_amp=tol, add_relative_noise=True
#     ),
#     CCE_ID="InverseCubic",
#     options_dict_user={
#         "Cce.Evolution.TimeStepper.AdamsBashforth.Order": 3,
#         "Cce.Evolution.StepChoosers.Constant": 0.1,
#         "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).AbsoluteTolerance": 1e-9*10,
#         "Cce.Evolution.StepChoosers.ErrorControl(SwshVars).RelativeTolerance": 1e-7*10,
#         "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).AbsoluteTolerance": 1e-9*10,
#         "Cce.Evolution.StepChoosers.ErrorControl(CoordVars).RelativeTolerance": 1e-8*10,
#         "Cce.LMax": 20,
#         "Cce.NumberOfRadialPoints": 15,
#         "Cce.ObservationLMax": 8,
#         "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MinOrder": 10,
#         "Cce.H5Interpolator.BarycentricRationalSpanInterpolator.MaxOrder": 10,
#         "Cce.H5LookaheadTimes": 10000,
#         "Cce.Filtering.RadialFilterHalfPower": 64,
#         "Cce.Filtering.RadialFilterAlpha": 35.0,
#         "Cce.Filtering.FilterLMax": 18,
#         "Cce.ScriInterpOrder": 5,
#         "Cce.ScriOutputDensity": 1,
#     }
# )


# Tests

In [ ]:
bondi_data_path = Path("/groups/sxs/hchaudha/scripts/CCE_read_and_modify_WT_data/del/original_data/BondiCceR0250.h5")
save_base_path = Path("/groups/sxs/hchaudha/scripts/CCE_read_and_modify_WT_data/del/modified_data")

In [ ]:
add_uniform_noise(bondi_data_path, save_base_path / "BondiCceR0250_noisy.h5", noise_amp=1e-15, add_relative_noise=True)
diff_dict = get_diff_WT_data(bondi_data_path, save_base_path / "BondiCceR0250_noisy.h5")

In [ ]:
remove_values_below_tolerance(
    bondi_data_path,
    save_base_path / "BondiCceR0250_filtered.h5",
    tolerance=1e-12,
    use_relative_tolerance=True,
)
diff_dict = get_diff_WT_data(
    bondi_data_path, save_base_path / "BondiCceR0250_filtered.h5"
)


In [ ]:
for key, value in diff_dict.items():
    print(f"{key}: {np.max(value[:,1:])}")

In [ ]:
# with h5py.File(bondi_data_path, "r") as f:
with h5py.File(save_base_path / "BondiCceR0250_filtered.h5", "r") as f:
    vars = [
        "Beta.dat",
        "DrJ.dat",
        "DuR.dat",
        "H.dat",
        "J.dat",
        "Q.dat",
        "R.dat",
        "U.dat",
        "W.dat",
    ]
    L2_vals = {}
    print(f.keys())
    for key in vars:
        print(f"{key}: {f[key].shape}")
        L2_vals[key] = np.max(f[key][2000:,1:],axis=1)
        # L2_vals[key] = np.array(f[key][2000:, 1:])
        # L2_vals[key] = np.linalg.norm(f[key][2000:,1:],axis=1)


In [ ]:
for key, value in L2_vals.items():
    if key != 'R.dat':
        continue
    # if key != 'H.dat':
    #     continue
    print(f"{key}: {value.shape}")
    plt.plot(value, label=key)
plt.xlabel('Time')
plt.yscale('log')
plt.ylabel('L2 Norm')
plt.legend()

In [ ]:
plt.plot(np.abs(L2_vals['J.dat'][1000,:]), label='R.dat')
plt.yscale('log')